In [1]:
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import cvxpy as cp
import random
import time
import os
from joblib import Parallel, delayed

from src.model import Nonneg_dagma, MetMulDagma, MetMulColide
import src.utils as utils

from baselines.colide import colide_ev, colide_nv
from baselines.dagma_linear import DAGMA_linear
from baselines.notears import notears_linear

SEED = 10
N_CPUS = os.cpu_count()

np.random.seed(SEED)
random.seed(SEED)

In [2]:
def get_lamb_value(n_nodes, n_samples, times=1):
    return np.sqrt(np.log(n_nodes) / n_samples) * times 

def run_exps(g, data_p, exps, thr=.2, verb=False):
    A_true, _, X = utils.simulate_sem(**data_p)
    A_true_bin = utils.to_bin(A_true, thr)
    X_std = utils.standarize(X)

    M, N = X.shape

    Z = X - X @ A_true
    fidelity = 1/data_p['n_samples']*la.norm(Z, 'fro')**2
    fidelity_norm = 1/data_p['n_samples']*la.norm(X_std - X_std @ A_true, 'fro')**2

    Sigma_hat = la.norm(Z, axis=0) / np.sqrt(M)

    print(f'{g}: Fidelity: {fidelity:.3f}  -  Fidelity (norm): {fidelity_norm:.3f}' +
          f'  -  Max Sigma: {np.max(Sigma_hat):.4f}  -  Min Sigma: {np.min(Sigma_hat):.4f}')

    shd, fscore, err, acyc, runtime = [np.zeros(len(exps))  for _ in range(5)]
    for i, exp in enumerate(exps):
        X_aux = X_std if 'standarize' in exp.keys() and exp['standarize'] else X

        args = exp['args'].copy()
        if 'fix_lamb' in exp.keys() and not exp['fix_lamb']:
            args['lamb'] = get_lamb_value(N, M, args['lamb'])

        if 'know_var' in exp.keys() and exp['know_var']:
            args['Sigma'] = data_p['var']

        if exp['model'] == notears_linear:
            t_i = time.time()
            A_est = notears_linear(X_aux, **args)
            t_solved = time.time() - t_i
        else:
            model = exp['model'](**exp['init']) if 'init' in exp.keys() else exp['model']()
            t_i = time.time()
            model.fit(X_aux, **args)
            t_solved = time.time() - t_i

            A_est = model.W_est

        A_est_bin = utils.to_bin(A_est, thr)
        shd[i], _, _ = utils.count_accuracy(A_true_bin, A_est_bin)
        fscore[i] = f1_score(A_true_bin.flatten(), A_est_bin.flatten())
        err[i] = utils.compute_norm_sq_err(A_true, A_est)
        acyc[i] = model.dagness(A_est) if hasattr(model, 'dagness') else 1
        runtime[i] = t_solved

        if verb and (g % N_CPUS == 0):
            print(f'\t-{g+1}: {exp["leg"]}: shd {shd[i]}  -  err: {err[i]:.3f}  -  time: {runtime[i]:.3f}')

    return shd, fscore, err, acyc, runtime

In [3]:
Exps = [
    # Method of Multipliers
    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 10000,
     'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-adam'},

    {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 10000,
     'iters_out': 10, 'beta': 5}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-adam-sf'},

    # {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 10000,
    #  'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista', 'restart': True}, 'standarize': False,
    #  'fix_lamb': False, 'leg': 'MM-fista-orig'},

    # {'model': MetMulDagma, 'args': {'stepsize': 1e-5, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 5000,
    #  'iters_out': 25, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista', 'restart': True}, 'standarize': False,
    #  'fix_lamb': False, 'leg': 'MM-fista-v1'},

    {'model': MetMulDagma, 'args': {'stepsize': 1e-5, 'alpha_0': .01, 'rho_0': .01, 's': 1, 'lamb': .2, 'iters_in': 5000,
     'iters_out': 50, 'beta': 1.5}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista', 'restart': True}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-fista'},

    {'model': MetMulDagma, 'args': {'stepsize': 1e-5, 'alpha_0': .01, 'rho_0': .01, 's': 1, 'lamb': .2, 'iters_in': 15000,
     'iters_out': 50, 'beta': 1.5}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista', 'restart': True}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-fista-sf1'},

    {'model': MetMulDagma, 'args': {'stepsize': 1e-5, 'alpha_0': .01, 'rho_0': .01, 's': 1, 'lamb': .2, 'iters_in': 20000,
     'iters_out': 50, 'beta': 1.5}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista', 'restart': True}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-fista-sf2'},

    {'model': MetMulDagma, 'args': {'stepsize': 1e-5, 'alpha_0': .01, 'rho_0': .01, 's': 1, 'lamb': .2, 'iters_in': 50000,
     'iters_out': 50, 'beta': 1.5}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista', 'restart': True}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-fista-sf3'},


    # {'model': MetMulDagma, 'args': {'stepsize': 1e-5, 'alpha_0': .01, 'rho_0': .01, 's': 1, 'lamb': .2, 'iters_in': 5000,
    #  'iters_out': 50, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista', 'restart': True}, 'standarize': False,
    #  'fix_lamb': False, 'leg': 'MM-fista-sf2'},

    # # COLIDE Cvx
    # {'model': MetMulColide, 'args': {'stepsize': 5e-5, 'alpha_0': .1, 'rho_0': .1, 's': 1, 'lamb': .1, 'iters_in': 30000,
    #  'iters_out': 10, 'beta': 1.2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'fista', 'restart': True}, 'standarize': False,
    #  'fix_lamb': False, 'leg': 'MM-Col-fista-r'},

    # {'model': MetMulColide, 'args': {'stepsize': 5e-5, 'alpha_0': .1, 'rho_0': .1, 's': 1, 'lamb': .1, 'iters_in': 30000,
    #  'iters_out': 10, 'beta': 1.2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam', 'restart': True}, 'standarize': False,
    #  'fix_lamb': False, 'leg': 'MM-Col-adam'},

    {'model': MetMulColide, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 20000,
     'iters_out': 10, 'beta': 2, 'sca_adam': True}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'sca'}, 'standarize': False,
     'fix_lamb': False, 'leg': 'MM-Col-sca'},


    # {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 10000,
    #  'iters_out': 10, 'beta': 5}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
    #  'fix_lamb': False, 'leg': 'MM-adam'},
    
    # {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 15000,
    #  'iters_out': 10, 'beta': 5}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
    #  'fix_lamb': False, 'leg': 'MM-adam'},


    # {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .1, 'iters_in': 15000,
    #  'iters_out': 15, 'beta': 5}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
    #  'fix_lamb': False, 'leg': 'MM-adam'},

    # ## 35 secs
    # {'model': MetMulDagma, 'args': {'stepsize': 3e-4, 'alpha_0': .01, 'rho_0': .05, 's': 1, 'lamb': .05, 'iters_in': 10000,
    #  'iters_out': 10, 'beta': 2}, 'init': {'acyclicity': 'logdet', 'primal_opt': 'adam'}, 'standarize': False,
    #  'fix_lamb': True, 'leg': 'MM-adam-fix'},



    # # Colide
    # {'model': colide_ev, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .7], 'warm_iter': 2e4,
    #  'max_iter': 7e4, 'lr': .0003}, 'standarize': False, 'leg': 'CoLiDE-EV'},
    
    # {'model': colide_nv, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .7], 'warm_iter': 2e4,
    #  'max_iter': 7e4, 'lr': .0003}, 'standarize': False, 'leg': 'CoLiDE-NV'},

    # DAGMA
    {'model': DAGMA_linear, 'init': {'loss_type': 'l2'}, 'args': {'lambda1': .05, 'T': 4, 's': [1.0, .9, .8, .8],
     'warm_iter': 2e4, 'max_iter': 7e4, 'lr': .0003}, 'standarize': False, 'leg': 'DAGMA'},

    # # NoTears
    # {'model': notears_linear, 'args': {'loss_type': 'l2', 'lambda1': .1, 'max_iter': 10}, 'standarize': False, 'leg': 'NoTears'},
]

# Comparing models - Homocedastic

### Var = 1

In [4]:
N = 100

n_dags = 25
verb = True
data_params = {
    'n_nodes': N,
    'n_samples': 1000, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),  # (.5, 1)
    'var': 1
}
print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 128


5: Fidelity: 100.449  -  Fidelity (norm): 380.409  -  Max Sigma: 1.0493  -  Min Sigma: 0.9527
10: Fidelity: 99.850  -  Fidelity (norm): 450.839  -  Max Sigma: 1.0457  -  Min Sigma: 0.9341
2: Fidelity: 100.379  -  Fidelity (norm): 468.797  -  Max Sigma: 1.0573  -  Min Sigma: 0.9320
3: Fidelity: 99.678  -  Fidelity (norm): 447.672  -  Max Sigma: 1.0530  -  Min Sigma: 0.9376
1: Fidelity: 99.770  -  Fidelity (norm): 284.436  -  Max Sigma: 1.0798  -  Min Sigma: 0.9415
8: Fidelity: 100.204  -  Fidelity (norm): 263.288  -  Max Sigma: 1.0482  -  Min Sigma: 0.9531
4: Fidelity: 99.680  -  Fidelity (norm): 357.563  -  Max Sigma: 1.0633  -  Min Sigma: 0.9389
24: Fidelity: 99.755  -  Fidelity (norm): 214.041  -  Max Sigma: 1.0604  -  Min Sigma: 0.9388
23: Fidelity: 99.537  -  Fidelity (norm): 358.840  -  Max Sigma: 1.0549  -  Min Sigma: 0.9396
0: Fidelity: 99.255  -  Fidelity (norm): 413.737  -  Max Sigma: 1.0533  -  Min Sigma: 0.9396
7: Fidelity: 99.987  -  Fidelity (norm): 261.413  -  Max Sigma: 

In [5]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,MM-adam,1.8400 ± 5.5619,0.9973 ± 0.0084,0.0077 ± 0.0084,0.0010 ± 0.0007,14.9181 ± 2.9733
1,MM-adam-sf,2.4000 ± 6.5299,0.9964 ± 0.0101,0.0084 ± 0.0097,0.0001 ± 0.0000,16.0110 ± 2.9763
2,MM-fista,1.0800 ± 3.8979,0.9981 ± 0.0062,0.0062 ± 0.0072,0.0000 ± 0.0000,11.6019 ± 1.2196
3,MM-fista-sf1,7.0400 ± 29.2198,0.9916 ± 0.0315,0.0165 ± 0.0474,0.0000 ± 0.0000,23.1269 ± 2.1854
4,MM-fista-sf2,1.2000 ± 1.6733,0.9977 ± 0.0030,0.0073 ± 0.0045,0.0000 ± 0.0000,27.4606 ± 4.7310
5,MM-fista-sf3,1.2800 ± 1.7325,0.9976 ± 0.0032,0.0076 ± 0.0049,0.0000 ± 0.0000,41.9690 ± 5.7932
6,MM-Col-sca,0.7200 ± 1.1496,0.9984 ± 0.0026,0.0069 ± 0.0038,0.0012 ± 0.0005,21.2219 ± 6.7701
7,DAGMA,25.9600 ± 11.7830,0.9584 ± 0.0163,0.0653 ± 0.0221,0.0005 ± 0.0001,15.8857 ± 2.8875


,leg,shd,fscore,err,acyc,time
0,MM-adam,0.0000 ± 5.5619,1.0000 ± 0.0084,0.0049 ± 0.0084,0.0010 ± 0.0007,15.5368 ± 2.9733
1,MM-adam-sf,0.0000 ± 6.5299,1.0000 ± 0.0101,0.0050 ± 0.0097,0.0001 ± 0.0000,16.1852 ± 2.9763
2,MM-fista,0.0000 ± 3.8979,1.0000 ± 0.0062,0.0044 ± 0.0072,0.0000 ± 0.0000,11.5272 ± 1.2196
3,MM-fista-sf1,0.0000 ± 29.2198,1.0000 ± 0.0315,0.0049 ± 0.0474,0.0000 ± 0.0000,22.9856 ± 2.1854
4,MM-fista-sf2,0.0000 ± 1.6733,1.0000 ± 0.0030,0.0045 ± 0.0045,0.0000 ± 0.0000,25.7329 ± 4.7310
5,MM-fista-sf3,0.0000 ± 1.7325,1.0000 ± 0.0032,0.0045 ± 0.0049,0.0000 ± 0.0000,40.6657 ± 5.7932
6,MM-Col-sca,0.0000 ± 1.1496,1.0000 ± 0.0026,0.0051 ± 0.0038,0.0010 ± 0.0005,18.6071 ± 6.7701
7,DAGMA,26.0000 ± 11.7830,0.9573 ± 0.0163,0.0659 ± 0.0221,0.0005 ± 0.0001,15.0175 ± 2.8875


### Var 1 - SF . 200 nodes

In [6]:
N = 200

n_dags = 25
verb = True
data_params = {
    'n_nodes': N,
    'n_samples': 5000, # 1000,
    'graph_type': 'sf',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),  # (.5, 1)
    'var': 1
}
print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 128
21: Fidelity: 200.699  -  Fidelity (norm): 770.945  -  Max Sigma: 1.0230  -  Min Sigma: 0.9722
17: Fidelity: 199.977  -  Fidelity (norm): 729.450  -  Max Sigma: 1.0256  -  Min Sigma: 0.9588
0: Fidelity: 200.039  -  Fidelity (norm): 748.872  -  Max Sigma: 1.0284  -  Min Sigma: 0.9749
22: Fidelity: 199.517  -  Fidelity (norm): 713.184  -  Max Sigma: 1.0276  -  Min Sigma: 0.9737
24: Fidelity: 199.736  -  Fidelity (norm): 744.764  -  Max Sigma: 1.0335  -  Min Sigma: 0.9701
19: Fidelity: 200.117  -  Fidelity (norm): 760.962  -  Max Sigma: 1.0264  -  Min Sigma: 0.9747
20: Fidelity: 199.703  -  Fidelity (norm): 718.840  -  Max Sigma: 1.0264  -  Min Sigma: 0.9745
4: Fidelity: 200.161  -  Fidelity (norm): 723.583  -  Max Sigma: 1.0251  -  Min Sigma: 0.9760
23: Fidelity: 200.176  -  Fidelity (norm): 708.259  -  Max Sigma: 1.0410  -  Min Sigma: 0.9722
12: Fidelity: 199.974  -  Fidelity (norm): 763.916  -  Max Sigma: 1.0331  -  Min Sigma: 0.9721
5: Fidelity: 200.219  -  Fidelity

In [7]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,MM-adam,22.5200 ± 41.0805,0.9843 ± 0.0241,0.0188 ± 0.0270,0.0001 ± 0.0001,145.1551 ± 42.8458
1,MM-adam-sf,70.0400 ± 96.2661,0.9550 ± 0.0498,0.0523 ± 0.0686,0.0000 ± 0.0000,147.6090 ± 45.6492
2,MM-fista,332.2800 ± 171.5563,0.7603 ± 0.1377,0.4180 ± 0.2102,0.0000 ± 0.0000,97.3156 ± 29.2035
3,MM-fista-sf1,57.9600 ± 61.9958,0.9604 ± 0.0412,0.0861 ± 0.0799,0.0001 ± 0.0003,176.6535 ± 109.7711
4,MM-fista-sf2,88.4800 ± 138.7548,0.9417 ± 0.0884,0.1051 ± 0.1463,0.0000 ± 0.0001,219.1964 ± 140.6102
5,MM-fista-sf3,33.0400 ± 73.1172,0.9771 ± 0.0480,0.0382 ± 0.0744,0.0000 ± 0.0001,388.1146 ± 319.8371
6,MM-Col-sca,1.4400 ± 2.4993,0.9987 ± 0.0022,0.0037 ± 0.0042,0.0005 ± 0.0002,367.9801 ± 158.7923
7,DAGMA,125.1200 ± 19.4058,0.9115 ± 0.0152,0.1692 ± 0.0216,0.0009 ± 0.0001,194.3880 ± 14.1682


,leg,shd,fscore,err,acyc,time
0,MM-adam,10.0000 ± 41.0805,0.9924 ± 0.0241,0.0109 ± 0.0270,0.0001 ± 0.0001,153.2232 ± 42.8458
1,MM-adam-sf,43.0000 ± 96.2661,0.9681 ± 0.0498,0.0329 ± 0.0686,0.0000 ± 0.0000,137.0525 ± 45.6492
2,MM-fista,346.0000 ± 171.5563,0.7638 ± 0.1377,0.4222 ± 0.2102,0.0000 ± 0.0000,93.9741 ± 29.2035
3,MM-fista-sf1,23.0000 ± 61.9958,0.9822 ± 0.0412,0.0393 ± 0.0799,0.0000 ± 0.0003,133.2636 ± 109.7711
4,MM-fista-sf2,17.0000 ± 138.7548,0.9854 ± 0.0884,0.0313 ± 0.1463,0.0000 ± 0.0001,193.1418 ± 140.6102
5,MM-fista-sf3,9.0000 ± 73.1172,0.9923 ± 0.0480,0.0109 ± 0.0744,0.0000 ± 0.0001,281.8538 ± 319.8371
6,MM-Col-sca,0.0000 ± 2.4993,1.0000 ± 0.0022,0.0016 ± 0.0042,0.0005 ± 0.0002,272.7742 ± 158.7923
7,DAGMA,125.0000 ± 19.4058,0.9122 ± 0.0152,0.1677 ± 0.0216,0.0009 ± 0.0001,197.1930 ± 14.1682


### Var 1 - SF

In [8]:
N = 100

n_dags = 25
verb = True
data_params = {
    'n_nodes': N,
    'n_samples': 1000, # 1000,
    'graph_type': 'sf',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),  # (.5, 1)
    'var': 1
}
print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 128
0: Fidelity: 99.565  -  Fidelity (norm): 347.911  -  Max Sigma: 1.0609  -  Min Sigma: 0.9413
13: Fidelity: 99.586  -  Fidelity (norm): 337.057  -  Max Sigma: 1.0567  -  Min Sigma: 0.9436
24: Fidelity: 101.126  -  Fidelity (norm): 336.172  -  Max Sigma: 1.0583  -  Min Sigma: 0.9534
16: Fidelity: 100.289  -  Fidelity (norm): 347.755  -  Max Sigma: 1.0600  -  Min Sigma: 0.9312
4: Fidelity: 99.496  -  Fidelity (norm): 336.679  -  Max Sigma: 1.0691  -  Min Sigma: 0.9528
19: Fidelity: 100.156  -  Fidelity (norm): 367.351  -  Max Sigma: 1.0621  -  Min Sigma: 0.9439
6: Fidelity: 99.782  -  Fidelity (norm): 335.791  -  Max Sigma: 1.0548  -  Min Sigma: 0.9574
3: Fidelity: 99.998  -  Fidelity (norm): 329.363  -  Max Sigma: 1.0570  -  Min Sigma: 0.9432
2: Fidelity: 100.067  -  Fidelity (norm): 352.894  -  Max Sigma: 1.0469  -  Min Sigma: 0.9461
9: Fidelity: 100.596  -  Fidelity (norm): 340.523  -  Max Sigma: 1.0503  -  Min Sigma: 0.9471
23: Fidelity: 100.970  -  Fidelity (norm):

In [9]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,MM-adam,8.7200 ± 8.7522,0.9859 ± 0.0135,0.0268 ± 0.0151,0.0007 ± 0.0003,20.4952 ± 5.3778
1,MM-adam-sf,19.4800 ± 13.0449,0.9697 ± 0.0190,0.0427 ± 0.0205,0.0000 ± 0.0000,19.8176 ± 5.3979
2,MM-fista,61.8000 ± 82.0517,0.9111 ± 0.1233,0.1552 ± 0.1921,0.0000 ± 0.0000,16.8595 ± 5.3690
3,MM-fista-sf1,15.6800 ± 27.4878,0.9768 ± 0.0353,0.0467 ± 0.0572,0.0000 ± 0.0000,35.6993 ± 12.3001
4,MM-fista-sf2,13.6000 ± 21.0694,0.9798 ± 0.0285,0.0430 ± 0.0495,0.0000 ± 0.0000,47.1903 ± 21.4443
5,MM-fista-sf3,7.1600 ± 9.3067,0.9884 ± 0.0145,0.0290 ± 0.0242,0.0000 ± 0.0001,83.3124 ± 54.5442
6,MM-Col-sca,3.8800 ± 3.9731,0.9932 ± 0.0066,0.0183 ± 0.0127,0.0018 ± 0.0005,98.0755 ± 33.7506
7,DAGMA,75.8800 ± 12.2811,0.8912 ± 0.0188,0.1968 ± 0.0320,0.0004 ± 0.0001,30.6719 ± 4.3330


,leg,shd,fscore,err,acyc,time
0,MM-adam,6.0000 ± 8.7522,0.9896 ± 0.0135,0.0233 ± 0.0151,0.0007 ± 0.0003,20.7487 ± 5.3778
1,MM-adam-sf,16.0000 ± 13.0449,0.9742 ± 0.0190,0.0353 ± 0.0205,0.0000 ± 0.0000,19.7321 ± 5.3979
2,MM-fista,35.0000 ± 82.0517,0.9503 ± 0.1233,0.0946 ± 0.1921,0.0000 ± 0.0000,15.5848 ± 5.3690
3,MM-fista-sf1,9.0000 ± 27.4878,0.9856 ± 0.0353,0.0343 ± 0.0572,0.0000 ± 0.0000,32.6992 ± 12.3001
4,MM-fista-sf2,8.0000 ± 21.0694,0.9870 ± 0.0285,0.0301 ± 0.0495,0.0000 ± 0.0000,42.2156 ± 21.4443
5,MM-fista-sf3,5.0000 ± 9.3067,0.9922 ± 0.0145,0.0227 ± 0.0242,0.0000 ± 0.0001,66.1708 ± 54.5442
6,MM-Col-sca,3.0000 ± 3.9731,0.9935 ± 0.0066,0.0131 ± 0.0127,0.0017 ± 0.0005,110.8069 ± 33.7506
7,DAGMA,75.0000 ± 12.2811,0.8901 ± 0.0188,0.2022 ± 0.0320,0.0004 ± 0.0001,29.5799 ± 4.3330


### Var = 5

In [6]:
N = 100

n_dags = 25
verb = True
data_params = {
    'n_nodes': N,
    'n_samples': 1000, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),  # (.5, 1)
    'var': 5**2
}
print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 128
0: Fidelity: 2489.122  -  Fidelity (norm): 270.319  -  Max Sigma: 5.3044  -  Min Sigma: 4.7064
2: Fidelity: 2489.122  -  Fidelity (norm): 349.104  -  Max Sigma: 5.3044  -  Min Sigma: 4.7064
3: Fidelity: 2489.122  -  Fidelity (norm): 534.642  -  Max Sigma: 5.3044  -  Min Sigma: 4.7064
7: Fidelity: 2489.122  -  Fidelity (norm): 247.888  -  Max Sigma: 5.3044  -  Min Sigma: 4.7064
6: Fidelity: 2489.122  -  Fidelity (norm): 388.458  -  Max Sigma: 5.3044  -  Min Sigma: 4.7064
10: Fidelity: 2489.122  -  Fidelity (norm): 305.040  -  Max Sigma: 5.3044  -  Min Sigma: 4.7064
1: Fidelity: 2489.122  -  Fidelity (norm): 386.075  -  Max Sigma: 5.3044  -  Min Sigma: 4.7064
8: Fidelity: 2489.122  -  Fidelity (norm): 354.969  -  Max Sigma: 5.3044  -  Min Sigma: 4.7064
11: Fidelity: 2489.122  -  Fidelity (norm): 320.739  -  Max Sigma: 5.3044  -  Min Sigma: 4.7064
4: Fidelity: 2489.122  -  Fidelity (norm): 431.347  -  Max Sigma: 5.3044  -  Min Sigma: 4.7064
5: Fidelity: 2489.122  -  Fid

In [7]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,MM-adam,0.6800 ± 1.2875,0.9986 ± 0.0028,0.0126 ± 0.0026,0.0234 ± 0.0210,32.7790 ± 3.8178
1,MM-fista-orig,13.0400 ± 57.3951,0.9770 ± 0.1002,0.0502 ± 0.1727,0.0395 ± 0.0712,23.2450 ± 5.0769
2,MM-fista-v1,3.7600 ± 9.7253,0.9942 ± 0.0134,0.0178 ± 0.0135,0.0001 ± 0.0002,14.5272 ± 3.8793
3,MM-fista-v2,9.6800 ± 34.7721,0.9877 ± 0.0371,0.0266 ± 0.0500,0.0000 ± 0.0000,27.6545 ± 5.9250
4,MM-Col-fista-r,76.9600 ± 109.6323,0.8665 ± 0.2130,0.2013 ± 0.2829,1.5470 ± 3.2960,69.4760 ± 56.4593
5,MM-Col-adam,16.9200 ± 14.9798,0.9728 ± 0.0205,0.0475 ± 0.0312,0.1174 ± 0.0632,60.4328 ± 7.7476
6,MM-Col-sca,7.8800 ± 3.8296,0.9836 ± 0.0074,0.0314 ± 0.0107,0.0020 ± 0.0013,20.8812 ± 7.0428
7,CoLiDE-EV,43.7600 ± 14.9246,0.9374 ± 0.0203,0.1145 ± 0.0228,0.0039 ± 0.0004,20.9876 ± 3.8062
8,CoLiDE-NV,102.9200 ± 31.0701,0.8647 ± 0.0364,0.1987 ± 0.0428,1.0000 ± 0.0000,22.5939 ± 3.5037
9,DAGMA,105.9600 ± 25.0735,0.8646 ± 0.0284,0.2826 ± 0.0359,0.0579 ± 0.0022,16.2014 ± 2.1765


,leg,shd,fscore,err,acyc,time
0,MM-adam,0.0000 ± 1.2875,1.0000 ± 0.0028,0.0118 ± 0.0026,0.0133 ± 0.0210,32.9446 ± 3.8178
1,MM-fista-orig,0.0000 ± 57.3951,1.0000 ± 0.1002,0.0145 ± 0.1727,0.0057 ± 0.0712,22.9029 ± 5.0769
2,MM-fista-v1,0.0000 ± 9.7253,1.0000 ± 0.0134,0.0134 ± 0.0135,0.0000 ± 0.0002,14.6871 ± 3.8793
3,MM-fista-v2,1.0000 ± 34.7721,0.9976 ± 0.0371,0.0151 ± 0.0500,0.0000 ± 0.0000,25.9134 ± 5.9250
4,MM-Col-fista-r,24.0000 ± 109.6323,0.9694 ± 0.2130,0.0558 ± 0.2829,0.0000 ± 3.2960,38.4779 ± 56.4593
5,MM-Col-adam,13.0000 ± 14.9798,0.9768 ± 0.0205,0.0418 ± 0.0312,0.1114 ± 0.0632,58.8199 ± 7.7476
6,MM-Col-sca,6.0000 ± 3.8296,0.9864 ± 0.0074,0.0286 ± 0.0107,0.0016 ± 0.0013,18.2870 ± 7.0428
7,CoLiDE-EV,41.0000 ± 14.9246,0.9382 ± 0.0203,0.1113 ± 0.0228,0.0040 ± 0.0004,20.5245 ± 3.8062
8,CoLiDE-NV,104.0000 ± 31.0701,0.8543 ± 0.0364,0.1954 ± 0.0428,1.0000 ± 0.0000,22.0155 ± 3.5037
9,DAGMA,105.0000 ± 25.0735,0.8680 ± 0.0284,0.2781 ± 0.0359,0.0577 ± 0.0022,16.3362 ± 2.1765


# Comparing models - Heterocedastic

In [4]:
N = 100

var = np.random.uniform(low=0.5, high=5.0, size=N)

n_dags = 25
verb = True
data_params = {
    'n_nodes': N,
    'n_samples': 1000, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),  # (.5, 1)
    'var': var**2
}
print('CPUs employed:', N_CPUS)  

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 128


2: Fidelity: 872.608  -  Fidelity (norm): 319.882  -  Max Sigma: 5.0152  -  Min Sigma: 0.5259
24: Fidelity: 863.756  -  Fidelity (norm): 263.323  -  Max Sigma: 4.9489  -  Min Sigma: 0.5434
13: Fidelity: 874.917  -  Fidelity (norm): 335.501  -  Max Sigma: 5.0021  -  Min Sigma: 0.5163
17: Fidelity: 878.207  -  Fidelity (norm): 244.702  -  Max Sigma: 4.9957  -  Min Sigma: 0.5071
1: Fidelity: 871.013  -  Fidelity (norm): 419.408  -  Max Sigma: 5.0638  -  Min Sigma: 0.5383
0: Fidelity: 870.844  -  Fidelity (norm): 450.452  -  Max Sigma: 4.9292  -  Min Sigma: 0.5192
8: Fidelity: 877.126  -  Fidelity (norm): 300.479  -  Max Sigma: 4.9606  -  Min Sigma: 0.5018
6: Fidelity: 872.243  -  Fidelity (norm): 244.535  -  Max Sigma: 5.0581  -  Min Sigma: 0.5090
7: Fidelity: 877.977  -  Fidelity (norm): 405.773  -  Max Sigma: 5.0410  -  Min Sigma: 0.5253
21: Fidelity: 871.764  -  Fidelity (norm): 383.183  -  Max Sigma: 4.9032  -  Min Sigma: 0.5208
11: Fidelity: 878.791  -  Fidelity (norm): 286.251  -  M

In [5]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,MM-adam,30.0400 ± 10.8866,0.9554 ± 0.0139,0.0745 ± 0.0204,0.0611 ± 0.0548,31.9663 ± 4.2499
1,MM-adam-sf,31.0800 ± 11.5928,0.9547 ± 0.0149,0.0750 ± 0.0208,0.0002 ± 0.0001,20.8470 ± 1.9850
2,MM-fista,68.8000 ± 65.3642,0.9005 ± 0.1333,0.2401 ± 0.1928,0.0000 ± 0.0000,22.6377 ± 10.3483
3,MM-fista-sf1,57.4000 ± 38.1733,0.9257 ± 0.0437,0.2110 ± 0.1253,0.0000 ± 0.0000,62.2734 ± 51.0052
4,MM-fista-sf2,64.0400 ± 69.3660,0.9036 ± 0.1433,0.2338 ± 0.2073,0.0000 ± 0.0000,85.2656 ± 53.2757
5,MM-fista-sf3,66.8400 ± 68.0445,0.9014 ± 0.1405,0.2462 ± 0.2000,0.0000 ± 0.0000,141.9105 ± 109.7173
6,MM-Col-sca,16.2400 ± 5.5805,0.9734 ± 0.0080,0.0554 ± 0.0175,0.0008 ± 0.0003,33.7900 ± 14.8088
7,DAGMA,353.6400 ± 43.9908,0.6041 ± 0.0392,0.7502 ± 0.0699,0.0132 ± 0.0016,24.3305 ± 4.8060


,leg,shd,fscore,err,acyc,time
0,MM-adam,30.0000 ± 10.8866,0.9557 ± 0.0139,0.0694 ± 0.0204,0.0771 ± 0.0548,32.2180 ± 4.2499
1,MM-adam-sf,30.0000 ± 11.5928,0.9548 ± 0.0149,0.0700 ± 0.0208,0.0001 ± 0.0001,21.0938 ± 1.9850
2,MM-fista,63.0000 ± 65.3642,0.9204 ± 0.1333,0.2293 ± 0.1928,-0.0000 ± 0.0000,18.1480 ± 10.3483
3,MM-fista-sf1,61.0000 ± 38.1733,0.9290 ± 0.0437,0.2291 ± 0.1253,-0.0000 ± 0.0000,50.8267 ± 51.0052
4,MM-fista-sf2,54.0000 ± 69.3660,0.9363 ± 0.1433,0.2225 ± 0.2073,-0.0000 ± 0.0000,67.7463 ± 53.2757
5,MM-fista-sf3,59.0000 ± 68.0445,0.9235 ± 0.1405,0.2324 ± 0.2000,-0.0000 ± 0.0000,99.2806 ± 109.7173
6,MM-Col-sca,16.0000 ± 5.5805,0.9735 ± 0.0080,0.0568 ± 0.0175,0.0009 ± 0.0003,26.5493 ± 14.8088
7,DAGMA,354.0000 ± 43.9908,0.6006 ± 0.0392,0.7423 ± 0.0699,0.0131 ± 0.0016,22.6544 ± 4.8060


# Comparing models - Standarized

In [10]:
for exp in Exps:
    exp['standarize'] = True

## 4N Edges

In [11]:
N = 100

n_dags = 20
verb=False
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 4*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),
    'var': 1
}

print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=verb) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 128
0: Fidelity: 99.166  -  Fidelity (norm): 287.659  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
12: Fidelity: 99.166  -  Fidelity (norm): 276.175  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
1: Fidelity: 99.166  -  Fidelity (norm): 327.727  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
3: Fidelity: 99.166  -  Fidelity (norm): 348.526  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
2: Fidelity: 99.166  -  Fidelity (norm): 389.500  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
4: Fidelity: 99.166  -  Fidelity (norm): 376.885  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
6: Fidelity: 99.166  -  Fidelity (norm): 331.765  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
7: Fidelity: 99.166  -  Fidelity (norm): 341.271  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
8: Fidelity: 99.166  -  Fidelity (norm): 326.439  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
16: Fidelity: 99.166  -  Fidelity (norm): 327.725  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
13: Fidelity: 99.166  -  Fidelity (norm): 342.814

In [12]:
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')  

,leg,shd,fscore,err,acyc,time
0,MM-adam,270.1000 ± 32.4991,0.4859 ± 0.0685,0.8261 ± 0.1548,0.0113 ± 0.0110,28.0367 ± 6.5598
1,MM-fista-orig,249.0000 ± 27.1993,0.5409 ± 0.0522,0.7027 ± 0.1076,0.0064 ± 0.0081,19.5006 ± 5.5160
2,MM-fista-v1,258.5500 ± 29.4525,0.5164 ± 0.0606,0.6961 ± 0.1076,0.0000 ± 0.0001,8.8757 ± 0.6442
3,MM-fista-v2,250.3000 ± 27.1018,0.5390 ± 0.0533,0.6701 ± 0.0981,0.0000 ± 0.0000,14.5071 ± 1.2073
4,MM-Col-fista-r,224.7500 ± 22.8339,0.6109 ± 0.0377,0.5265 ± 0.0661,0.0166 ± 0.0248,54.6571 ± 6.5221
5,MM-Col-adam,235.7500 ± 25.9593,0.5838 ± 0.0484,0.5796 ± 0.0911,0.0152 ± 0.0271,139.0550 ± 8.5189
6,MM-Col-sca,224.6500 ± 22.1501,0.6100 ± 0.0360,0.5311 ± 0.0580,0.0041 ± 0.0057,21.9450 ± 6.2489
7,CoLiDE-EV,472.8000 ± 21.5699,0.1574 ± 0.0220,1.7130 ± 0.0298,0.0000 ± 0.0000,18.6778 ± 2.4515
8,CoLiDE-NV,349.5500 ± 21.8048,0.3302 ± 0.0351,1.2432 ± 0.0741,1.0000 ± 0.0000,21.3536 ± 2.7936
9,DAGMA,373.5500 ± 24.3423,0.2612 ± 0.0339,1.4271 ± 0.0655,0.0000 ± 0.0000,17.6749 ± 2.9891


,leg,shd,fscore,err,acyc,time
0,MM-adam,280.0000 ± 32.4991,0.4753 ± 0.0685,0.8370 ± 0.1548,0.0075 ± 0.0110,29.5925 ± 6.5598
1,MM-fista-orig,255.5000 ± 27.1993,0.5226 ± 0.0522,0.6969 ± 0.1076,0.0014 ± 0.0081,18.9727 ± 5.5160
2,MM-fista-v1,266.5000 ± 29.4525,0.5000 ± 0.0606,0.7014 ± 0.1076,0.0000 ± 0.0001,8.7025 ± 0.6442
3,MM-fista-v2,253.5000 ± 27.1018,0.5378 ± 0.0533,0.6798 ± 0.0981,0.0000 ± 0.0000,15.0143 ± 1.2073
4,MM-Col-fista-r,228.0000 ± 22.8339,0.6089 ± 0.0377,0.5353 ± 0.0661,0.0056 ± 0.0248,54.7012 ± 6.5221
5,MM-Col-adam,239.5000 ± 25.9593,0.5800 ± 0.0484,0.5990 ± 0.0911,0.0044 ± 0.0271,135.8324 ± 8.5189
6,MM-Col-sca,229.0000 ± 22.1501,0.5997 ± 0.0360,0.5389 ± 0.0580,0.0015 ± 0.0057,22.2301 ± 6.2489
7,CoLiDE-EV,475.0000 ± 21.5699,0.1597 ± 0.0220,1.7163 ± 0.0298,0.0000 ± 0.0000,18.5080 ± 2.4515
8,CoLiDE-NV,348.5000 ± 21.8048,0.3336 ± 0.0351,1.2452 ± 0.0741,1.0000 ± 0.0000,21.2817 ± 2.7936
9,DAGMA,376.0000 ± 24.3423,0.2622 ± 0.0339,1.4352 ± 0.0655,0.0000 ± 0.0000,17.7808 ± 2.9891


In [13]:
###  1000 samples - 100 nodes  #####
N = 100

n_dags = 20
data_params = {
    'n_nodes': N,
    'n_samples': 500, # 1000,
    'graph_type': 'er',
    'edges': 2*N,
    'edge_type': 'positive',
    'w_range': (.5, 1),
    'var': 1,
}

print('CPUs employed:', N_CPUS)

t_init = time.time()
results = Parallel(n_jobs=N_CPUS)(delayed(run_exps)(g, data_params, Exps, thr=.2, verb=False) for g in range(n_dags))
ellapsed_time = (time.time() - t_init)/60
print(f'----- Solved in {ellapsed_time:.3f} minutes -----')

# Extract results
shd, fscore, err, acyc, runtime = zip(*results)
metrics = {'shd': shd, 'fscore': fscore, 'err': err, 'acyc': acyc, 'time': runtime}


CPUs employed: 128
0: Fidelity: 99.166  -  Fidelity (norm): 82.372  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
1: Fidelity: 99.166  -  Fidelity (norm): 82.195  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
2: Fidelity: 99.166  -  Fidelity (norm): 85.115  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
18: Fidelity: 99.166  -  Fidelity (norm): 82.515  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
14: Fidelity: 99.166  -  Fidelity (norm): 77.961  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
16: Fidelity: 99.166  -  Fidelity (norm): 89.170  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
12: Fidelity: 99.166  -  Fidelity (norm): 82.132  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
13: Fidelity: 99.166  -  Fidelity (norm): 79.054  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
6: Fidelity: 99.166  -  Fidelity (norm): 78.458  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
15: Fidelity: 99.166  -  Fidelity (norm): 79.855  -  Max Sigma: 1.0720  -  Min Sigma: 0.9029
3: Fidelity: 99.166  -  Fidelity (norm): 82.516  -  Max

In [14]:
# M = 1000
exps_leg = [exp['leg'] for exp in Exps]
utils.display_results(exps_leg, metrics, agg='mean')
utils.display_results(exps_leg, metrics, agg='median')

,leg,shd,fscore,err,acyc,time
0,MM-adam,50.3000 ± 11.8874,0.8244 ± 0.0354,0.3875 ± 0.0543,0.0064 ± 0.0066,18.1276 ± 3.8541
1,MM-fista-orig,50.4500 ± 12.7337,0.8219 ± 0.0400,0.3918 ± 0.0572,0.0075 ± 0.0071,15.7892 ± 3.9159
2,MM-fista-v1,49.4500 ± 12.7651,0.8287 ± 0.0368,0.3712 ± 0.0489,0.0001 ± 0.0001,8.5438 ± 0.8121
3,MM-fista-v2,46.8000 ± 11.0842,0.8364 ± 0.0332,0.3513 ± 0.0499,0.0000 ± 0.0000,14.2025 ± 1.1988
4,MM-Col-fista-r,37.6000 ± 12.1918,0.8786 ± 0.0369,0.2963 ± 0.0643,0.0180 ± 0.0153,43.8494 ± 6.1315
5,MM-Col-adam,37.3000 ± 12.7832,0.8788 ± 0.0401,0.2936 ± 0.0652,0.0199 ± 0.0203,125.4067 ± 4.3700
6,MM-Col-sca,38.1000 ± 11.7213,0.8745 ± 0.0352,0.3031 ± 0.0607,0.0055 ± 0.0060,13.7107 ± 1.4077
7,CoLiDE-EV,226.8500 ± 17.8305,0.3076 ± 0.0331,1.4413 ± 0.0732,0.0000 ± 0.0000,16.0092 ± 1.7218
8,CoLiDE-NV,162.1500 ± 18.3311,0.4716 ± 0.0519,1.1425 ± 0.1011,1.0000 ± 0.0000,18.3627 ± 1.8998
9,DAGMA,146.2000 ± 15.8827,0.5047 ± 0.0444,1.0233 ± 0.0870,0.0000 ± 0.0000,14.4935 ± 2.3529


,leg,shd,fscore,err,acyc,time
0,MM-adam,46.5000 ± 11.8874,0.8206 ± 0.0354,0.3885 ± 0.0543,0.0031 ± 0.0066,16.6590 ± 3.8541
1,MM-fista-orig,46.5000 ± 12.7337,0.8270 ± 0.0400,0.3893 ± 0.0572,0.0043 ± 0.0071,15.7908 ± 3.9159
2,MM-fista-v1,47.0000 ± 12.7651,0.8279 ± 0.0368,0.3661 ± 0.0489,0.0000 ± 0.0001,8.6129 ± 0.8121
3,MM-fista-v2,46.0000 ± 11.0842,0.8363 ± 0.0332,0.3529 ± 0.0499,0.0000 ± 0.0000,14.2503 ± 1.1988
4,MM-Col-fista-r,37.5000 ± 12.1918,0.8746 ± 0.0369,0.3097 ± 0.0643,0.0125 ± 0.0153,43.7839 ± 6.1315
5,MM-Col-adam,35.0000 ± 12.7832,0.8873 ± 0.0401,0.2867 ± 0.0652,0.0130 ± 0.0203,124.9219 ± 4.3700
6,MM-Col-sca,38.0000 ± 11.7213,0.8725 ± 0.0352,0.3032 ± 0.0607,0.0038 ± 0.0060,13.2666 ± 1.4077
7,CoLiDE-EV,228.5000 ± 17.8305,0.3182 ± 0.0331,1.4266 ± 0.0732,0.0000 ± 0.0000,15.8861 ± 1.7218
8,CoLiDE-NV,160.0000 ± 18.3311,0.4686 ± 0.0519,1.1439 ± 0.1011,1.0000 ± 0.0000,18.4459 ± 1.8998
9,DAGMA,143.5000 ± 15.8827,0.5012 ± 0.0444,1.0023 ± 0.0870,0.0000 ± 0.0000,14.3761 ± 2.3529
